In [1]:
# Add "parameters" (jupyter-notebook) tag to this cell, to allow papermill to inject different parameters
from datetime import date
it=0  #Iteration of gridsearch
# to put it all in one folder by date, will be replaced by papermill
today=date.today() 
rdate=today.strftime("%Y-%m-%d")
rdate="2021-08-31"
# Training Params
best_runs = []
training_stats = {}
epochs = 15
loops = 5

In [2]:
# Parameters
it = 6
rdate = "2021-08-31"


In [3]:
# See if running on Colab (for setting the correct workdir and installing all dependencies)
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  g_colab = True
else:
  print('Not running on CoLab')
  g_colab = False

Not running on CoLab


In [4]:
# connect to drive
if g_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/1 Job/Product and Code/CogAlex 2.0/"

# Choose model
# Gridsearch parameters
from sklearn.model_selection import ParameterGrid

# Original results with "xlm_roberta_base"
grid = [{"model_name": ["xlm-roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new"], ["de_train_new"], ["de_val_new"],
                      ["en_train_new", "en_val_new"], ["en_train_new"], ["en_val_new"], 
                      ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]},
        {"model_name": ["xlm-roberta-large", "distilbert-base-multilingual-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased", "roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]}]

pg = list(ParameterGrid(grid))
print(len(pg))
model_name = pg[it]["model_name"]
datasets = pg[it]["datasets"]
print(pg[it])

18
{'datasets': ['en_val_new'], 'model_name': 'xlm-roberta-base'}


# Manually set parameters

In [5]:
# Savedir
import os
savedir = f"./{rdate}/averages/{model_name}_{it}_{datasets}"
if g_colab:
    workdir = os.getcwd()
else:
    workdir = os.getcwd().replace("/home/","/binfl/")
model_dir = f"{workdir}/saved_models/{rdate}"

# Check for GPU support
import torch
if torch.cuda.is_available():
        print("Using GPU for inference")
        print(torch.cuda.get_device_name(torch.cuda.current_device()))
        device = torch.cuda.current_device()
else:
        print("Using CPU for inference")
        device = -1

Using GPU for inference
GeForce GTX 1080


In [6]:
if g_colab:
    !pip install transformers
    !pip install sentencepiece

# Libraries

In [7]:
import torch                                              #for training the model
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd                                       #for handling the data
from transformers import XLMRobertaTokenizer, AutoTokenizer              #for loading the pretrained model and tokenizer
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from sklearn import preprocessing                         #for label encoding
from sklearn.metrics import classification_report         #for showing performance on validation/test sets
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid         #for grid search
from sklearn.model_selection import ParameterSampler      #for random search
from sklearn.utils.fixes import loguniform
import sentencepiece
import matplotlib.pyplot as plt
import time
import datetime
import random
import numpy as np
%matplotlib inline
import seaborn as sns
# For file saving etc.
import os
import shutil

# Load Data

In [8]:
# sk learn label encoder for changing the labels to integers
labels=["ANT", "HYP", "RANDOM", "SYN"]
le = preprocessing.LabelEncoder()
le.fit(labels)
print(labels)

['ANT', 'HYP', 'RANDOM', 'SYN']


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
if "gpt2" in model_name:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token_id

1

# Evaluate Validation

In [10]:
label_dict = {i : l for i,l in enumerate(labels)}
print(label_dict)

{0: 'ANT', 1: 'HYP', 2: 'RANDOM', 3: 'SYN'}


In [11]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "all":0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "all":""}
for run in os.listdir(model_dir):
    print(run + "\n")
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",
                                                               num_labels=4,  
                                                               id2label=label_dict)                                                        
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    validgold_zh, validgold_de, validgold_en = [open(f"./datasets/validgold_{l}_data.txt").readlines() for l in ["chinese", "german", "english"]]
    validgold_all =   validgold_zh + validgold_de + validgold_en 
    for data, read_lang in zip([validgold_zh, validgold_de, validgold_en, validgold_all], ["chinese", "german", "english", "all"]):
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict

Run 3



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.89      0.88      0.88       136
         HYP       0.90      0.86      0.88       145
      RANDOM       0.93      0.95      0.94       428
         SYN       0.84      0.85      0.85       129

    accuracy                           0.91       838
   macro avg       0.89      0.88      0.89       838
weighted avg       0.91      0.91      0.91       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.93      0.88      0.90       136
         HYP       0.95      0.86      0.90       145
         SYN       0.90      0.85      0.88       129

   micro avg       0.93      0.86      0.89       410
   macro avg       0.93      0.86      0.89       410
weighted avg       0.93      0.86      0.89       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.48      0.45      0.46       275
         HYP       0.47      0.45      0.46       294
      RANDOM       0.72      0.76      0.74       786
         SYN       0.41      0.38      0.40       272

    accuracy                           0.59      1627
   macro avg       0.52      0.51      0.52      1627
weighted avg       0.58      0.59      0.59      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.45      0.55       275
         HYP       0.57      0.45      0.50       294
         SYN       0.52      0.38      0.44       272

   micro avg       0.59      0.43      0.50       841
   macro avg       0.60      0.43      0.50       841
weighted avg       0.60      0.43      0.50       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.56      0.50      0.53       308
         HYP       0.48      0.50      0.49       292
      RANDOM       0.78      0.81      0.80       877
         SYN       0.41      0.40      0.40       259

    accuracy                           0.64      1736
   macro avg       0.56      0.55      0.55      1736
weighted avg       0.64      0.64      0.64      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.75      0.50      0.60       308
         HYP       0.58      0.50      0.54       292
         SYN       0.50      0.40      0.44       259

   micro avg       0.61      0.47      0.53       859
   macro avg       0.61      0.47      0.53       859
weighted avg       0.62      0.47      0.53       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.59      0.55      0.57       719
         HYP       0.55      0.55      0.55       731
      RANDOM       0.79      0.82      0.80      2091
         SYN       0.50      0.48      0.49       660

    accuracy                           0.67      4201
   macro avg       0.61      0.60      0.60      4201
weighted avg       0.67      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.78      0.55      0.64       719
         HYP       0.65      0.55      0.60       731
         SYN       0.60      0.48      0.53       660

   micro avg       0.68      0.53      0.59      2110
   macro avg       0.68      0.53      0.59      2110
weighted avg       0.68      0.53      0.59      2110

Run 5



...

Results for all: 

              precision    recall  f1-score   support

         ANT       0.59      0.55      0.57       719
         HYP       0.55      0.55      0.55       731
      RANDOM       0.79      0.82      0.80      2091
         SYN       0.50      0.48      0.49       660

    accuracy                           0.67      4201
   macro avg       0.61      0.60      0.60      4201
weighted avg       0.67      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.78      0.55      0.64       719
         HYP       0.65      0.55      0.60       731
         SYN       0.60      0.48      0.53       660

   micro avg       0.68      0.53      0.59      2110
   macro avg       0.68      0.53      0.59      2110
weighted avg       0.68      0.53      0.59      2110

Run 1



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.86      0.92      0.89       136
         HYP       0.89      0.87      0.88       145
      RANDOM       0.94      0.95      0.95       428
         SYN       0.89      0.84      0.87       129

    accuracy                           0.91       838
   macro avg       0.90      0.89      0.90       838
weighted avg       0.91      0.91      0.91       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.93      0.92      0.92       136
         HYP       0.95      0.87      0.91       145
         SYN       0.92      0.84      0.88       129

   micro avg       0.93      0.88      0.90       410
   macro avg       0.93      0.88      0.90       410
weighted avg       0.93      0.88      0.90       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.45      0.47      0.46       275
         HYP       0.45      0.50      0.47       294
      RANDOM       0.74      0.73      0.73       786
         SYN       0.44      0.39      0.41       272

    accuracy                           0.59      1627
   macro avg       0.52      0.52      0.52      1627
weighted avg       0.59      0.59      0.59      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.67      0.47      0.55       275
         HYP       0.58      0.50      0.54       294
         SYN       0.55      0.39      0.45       272

   micro avg       0.60      0.45      0.51       841
   macro avg       0.60      0.45      0.51       841
weighted avg       0.60      0.45      0.51       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.53      0.50      0.51       308
         HYP       0.46      0.52      0.49       292
      RANDOM       0.78      0.77      0.78       877
         SYN       0.40      0.37      0.39       259

    accuracy                           0.62      1736
   macro avg       0.54      0.54      0.54      1736
weighted avg       0.62      0.62      0.62      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.75      0.50      0.60       308
         HYP       0.58      0.52      0.55       292
         SYN       0.48      0.37      0.42       259

   micro avg       0.60      0.47      0.53       859
   macro avg       0.60      0.47      0.52       859
weighted avg       0.61      0.47      0.53       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.57      0.56      0.57       719
         HYP       0.53      0.58      0.55       731
      RANDOM       0.80      0.79      0.80      2091
         SYN       0.51      0.47      0.49       660

    accuracy                           0.67      4201
   macro avg       0.60      0.60      0.60      4201
weighted avg       0.67      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.77      0.56      0.65       719
         HYP       0.65      0.58      0.62       731
         SYN       0.61      0.47      0.53       660

   micro avg       0.68      0.54      0.60      2110
   macro avg       0.68      0.54      0.60      2110
weighted avg       0.68      0.54      0.60      2110



In [12]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results.loc[k] = temp_df.mean()
avg_val_results.update(avg_val_results.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results

,precision,recall,f1-score,support
chinese,90.7,90.7,90.7,838.0
german,58.3,59.0,58.6,1627.0
english,63.4,63.7,63.5,1736.0
all,66.9,67.2,67.0,4201.0


In [13]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results_no_random.loc[k] = temp_df.mean()
avg_val_results_no_random.update(avg_val_results_no_random.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results_no_random

,precision,recall,f1-score,support
chinese,92.8,86.4,89.5,410.0
german,59.7,43.3,50.1,841.0
english,61.6,46.9,53.1,859.0
all,67.9,53.1,59.6,2110.0


In [14]:
# save average scores
os.makedirs(savedir, exist_ok=True)
val_preds_savedir = savedir + "/val_preds"
os.makedirs(val_preds_savedir, exist_ok=True)
avg_val_results.to_csv(f"{savedir}/val_avg_{model_name}_{it}.csv")
avg_val_results_no_random.to_csv(f"{savedir}/val_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(val_preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        

# Evaluate on Gold

In [15]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "italian":0, "german NEW":0, "english NEW": 0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "italian":"", "german NEW":"", "english NEW": ""}
for run in os.listdir(model_dir):
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",  id2label=label_dict)
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    gold_zh, gold_de, gold_en, gold_it = [open(f"./datasets/gold_{l}_data.txt").readlines() for l in ["chinese", "german", "english", "italian"]]
    gold_de_new, gold_en_new = [open(f"./datasets/gold_{l}_data_new.txt").readlines() for l in ["german", "english"]]   
    for data, read_lang in zip([gold_zh, gold_de, gold_en, gold_it, gold_de_new, gold_en_new], ["chinese", "german", "english", "italian", "german NEW", "english NEW"]):
        # Load data for prediction/scoring
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        
        #Predict labels on test_set
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict
        #print(results_no_random[run][read_lang]["weighted avg"])

Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.89      0.89      0.89       142
         HYP       0.88      0.81      0.84       129
      RANDOM       0.94      0.96      0.95       445
         SYN       0.83      0.83      0.83       122

    accuracy                           0.91       838
   macro avg       0.88      0.87      0.88       838
weighted avg       0.91      0.91      0.91       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.95      0.89      0.92       142
         HYP       0.91      0.81      0.86       129
         SYN       0.86      0.83      0.85       122

   micro avg       0.91      0.85      0.88       393
   macro avg       0.91      0.85      0.88       393
weighted avg       0.91      0.85      0.88       393



Results for german: 

              precision    recall  f1-score   support

         ANT       0.50      0.42      0.46       281
         HYP       0.47      0.49      0.48       286
      RANDOM       0.75      0.79      0.77       796
         SYN       0.37      0.35      0.36       265

    accuracy                           0.60      1628
   macro avg       0.52      0.51      0.52      1628
weighted avg       0.59      0.60      0.60      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.67      0.42      0.52       281
         HYP       0.59      0.49      0.54       286
         SYN       0.46      0.35      0.40       265

   micro avg       0.57      0.42      0.49       832
   macro avg       0.57      0.42      0.48       832
weighted avg       0.58      0.42      0.49       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.55      0.55      0.55       306
         HYP       0.47      0.48      0.48       279
      RANDOM       0.81      0.82      0.81       887
         SYN       0.44      0.42      0.43       266

    accuracy                           0.65      1738
   macro avg       0.57      0.57      0.57      1738
weighted avg       0.65      0.65      0.65      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.73      0.55      0.63       306
         HYP       0.57      0.48      0.52       279
         SYN       0.53      0.42      0.47       266

   micro avg       0.61      0.49      0.54       851
   macro avg       0.61      0.48      0.54       851
weighted avg       0.62      0.49      0.54       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.31      0.38      0.34       144
         HYP       0.42      0.35      0.38       153
      RANDOM       0.78      0.68      0.73       523
         SYN       0.36      0.47      0.40       187

    accuracy                           0.55      1007
   macro avg       0.46      0.47      0.46      1007
weighted avg       0.58      0.55      0.56      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.51      0.38      0.43       144
         HYP       0.55      0.35      0.43       153
         SYN       0.49      0.47      0.48       187

   micro avg       0.51      0.40      0.45       484
   macro avg       0.52      0.40      0.45       484
weighted avg       0.52      0.40      0.45       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.59      0.50      0.54       281
         HYP       0.52      0.56      0.54       286
      RANDOM       0.78      0.82      0.80       796
         SYN       0.42      0.40      0.41       265

    accuracy                           0.65      1628
   macro avg       0.58      0.57      0.57      1628
weighted avg       0.64      0.65      0.65      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.75      0.50      0.60       281
         HYP       0.63      0.56      0.59       286
         SYN       0.50      0.40      0.45       265

   micro avg       0.62      0.49      0.55       832
   macro avg       0.63      0.49      0.55       832
weighted avg       0.63      0.49      0.55       832



...

Results for german: 

              precision    recall  f1-score   support

         ANT       0.47      0.44      0.45       281
         HYP       0.43      0.53      0.48       286
      RANDOM       0.76      0.74      0.75       796
         SYN       0.38      0.34      0.36       265

    accuracy                           0.59      1628
   macro avg       0.51      0.51      0.51      1628
weighted avg       0.59      0.59      0.59      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.66      0.44      0.53       281
         HYP       0.57      0.53      0.55       286
         SYN       0.46      0.34      0.39       265

   micro avg       0.56      0.44      0.49       832
   macro avg       0.56      0.44      0.49       832
weighted avg       0.57      0.44      0.49       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.52      0.56      0.54       306
         HYP       0.44      0.48      0.46       279
      RANDOM       0.81      0.81      0.81       887
         SYN       0.42      0.35      0.38       266

    accuracy                           0.64      1738
   macro avg       0.55      0.55      0.55      1738
weighted avg       0.64      0.64      0.64      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.69      0.56      0.62       306
         HYP       0.53      0.48      0.51       279
         SYN       0.51      0.35      0.42       266

   micro avg       0.58      0.47      0.52       851
   macro avg       0.58      0.47      0.51       851
weighted avg       0.58      0.47      0.52       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.40      0.46      0.43       144
         HYP       0.40      0.43      0.42       153
      RANDOM       0.79      0.67      0.72       523
         SYN       0.34      0.43      0.38       187

    accuracy                           0.56      1007
   macro avg       0.48      0.50      0.49      1007
weighted avg       0.59      0.56      0.57      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.60      0.46      0.52       144
         HYP       0.55      0.43      0.49       153
         SYN       0.50      0.43      0.46       187

   micro avg       0.54      0.44      0.49       484
   macro avg       0.55      0.44      0.49       484
weighted avg       0.55      0.44      0.49       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.52      0.49      0.51       281
         HYP       0.46      0.62      0.53       286
      RANDOM       0.80      0.76      0.78       796
         SYN       0.42      0.35      0.38       265

    accuracy                           0.62      1628
   macro avg       0.55      0.55      0.55      1628
weighted avg       0.63      0.62      0.62      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.49      0.58       281
         HYP       0.59      0.62      0.60       286
         SYN       0.50      0.35      0.41       265

   micro avg       0.60      0.49      0.54       832
   macro avg       0.60      0.48      0.53       832
weighted avg       0.60      0.49      0.53       832



Results for english NEW: 

              precision    recall  f1-score   support

         ANT       0.57      0.60      0.58       306
         HYP       0.49      0.60      0.54       283
      RANDOM       0.85      0.82      0.83       887
         SYN       0.49      0.42      0.45       262

    accuracy                           0.68      1738
   macro avg       0.60      0.61      0.60      1738
weighted avg       0.69      0.68      0.68      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.74      0.60      0.66       306
         HYP       0.58      0.60      0.59       283
         SYN       0.59      0.42      0.49       262

   micro avg       0.64      0.54      0.59       851
   macro avg       0.64      0.54      0.58       851
weighted avg       0.64      0.54      0.58       851



In [16]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results.loc[k] = temp_df.mean()
avg_test_results.update(avg_test_results.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results

,precision,recall,f1-score,support
chinese,90.7,90.8,90.7,838.0
german,59.4,60.0,59.6,1628.0
english,65.0,65.2,65.1,1738.0
italian,57.9,54.9,56.0,1007.0
german NEW,64.1,64.4,64.1,1628.0
english NEW,69.3,68.8,69.0,1738.0


In [17]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results_no_random.loc[k] = temp_df.mean()
avg_test_results_no_random.update(avg_test_results_no_random.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results_no_random

,precision,recall,f1-score,support
chinese,90.9,84.8,87.7,393.0
german,57.4,42.6,48.7,832.0
english,60.8,48.3,53.8,851.0
italian,52.2,41.0,45.7,484.0
german NEW,62.4,48.8,54.5,832.0
english NEW,65.5,54.8,59.5,851.0


In [18]:
# save average scores
os.makedirs(savedir, exist_ok=True)
preds_savedir = savedir + "/preds"
os.makedirs(preds_savedir, exist_ok=True)
avg_test_results.to_csv(f"{savedir}/test_avg_{model_name}_{it}.csv")
avg_test_results_no_random.to_csv(f"{savedir}/test_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        